## **📄 Dynamic FloSports Case Study Builder (Fallback-Ready)**






In [1]:
# build_case_study.py

import os
from datetime import datetime
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

# ------------ CONFIG ------------
PROJECT_FOLDER = "./output"  # where your phase results are stored
OUTPUT_FILE = os.path.join(PROJECT_FOLDER, "FloSports_Case_Study.pdf")
PHASES = [
    ("Phase 1: Project Setup", "phase1_summary.txt", "phase1_chart.png"),
    ("Phase 2: Exploratory Data Analysis", "phase2_summary.txt", "phase2_chart.png"),
    ("Phase 3: A/B Testing", "phase3_summary.txt", "phase3_chart.png"),
    ("Phase 4: Demand Forecasting", "phase4_summary.txt", "phase4_chart.png"),
    ("Phase 5: Survey Analysis", "phase5_summary.txt", "phase5_chart.png"),
]
# ------------ CONFIG ------------

# Create styles
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name="TitleStyle", fontSize=22, leading=26, textColor=colors.HexColor("#0A2342"), spaceAfter=20))
styles.add(ParagraphStyle(name="SubTitleStyle", fontSize=16, leading=20, textColor=colors.HexColor("#00A7A7"), spaceAfter=10))
styles.add(ParagraphStyle(name="BodyStyle", fontSize=11, leading=15, spaceAfter=10))
styles.add(ParagraphStyle(name="MissingStyle", fontSize=11, leading=15, textColor=colors.red, spaceAfter=10))
styles.add(ParagraphStyle(name="HeaderStyle", fontSize=14, leading=18, textColor=colors.HexColor("#F25C05"), spaceAfter=12, underlineWidth=1))

def load_text_file(file_path):
    """Load text file content, fallback if missing."""
    if os.path.exists(file_path):
        with open(file_path, encoding="utf-8") as f:
            return f.read()
    return None

def load_image(file_path, width=480):
    """Load image if exists, else return None."""
    if os.path.exists(file_path):
        return Image(file_path, width=width, height=width * 0.6)  # Maintain aspect ratio
    return None

def build_pdf():
    doc = SimpleDocTemplate(OUTPUT_FILE, pagesize=A4)
    elements = []

    # Title Page
    elements.append(Paragraph("FloSports Data Insights Case Study", styles["TitleStyle"]))
    elements.append(Paragraph("Portfolio Project by [Your Name]", styles["SubTitleStyle"]))
    elements.append(Paragraph(datetime.now().strftime("%B %d, %Y"), styles["BodyStyle"]))
    elements.append(Spacer(1, 50))

    # Executive Summary placeholder (from Phase 1 if available)
    phase1_summary = load_text_file(os.path.join(PROJECT_FOLDER, "phase1_summary.txt"))
    if phase1_summary:
        elements.append(Paragraph("Executive Summary", styles["HeaderStyle"]))
        elements.append(Paragraph(phase1_summary, styles["BodyStyle"]))
    else:
        elements.append(Paragraph("Executive Summary", styles["HeaderStyle"]))
        elements.append(Paragraph("No executive summary available yet.", styles["MissingStyle"]))
    elements.append(PageBreak())

    # Loop through each phase
    for title, summary_file, chart_file in PHASES:
        elements.append(Paragraph(title, styles["HeaderStyle"]))

        # Summary
        summary_path = os.path.join(PROJECT_FOLDER, summary_file)
        summary_text = load_text_file(summary_path)
        if summary_text:
            elements.append(Paragraph(summary_text, styles["BodyStyle"]))
        else:
            elements.append(Paragraph("No summary available for this phase.", styles["MissingStyle"]))

        # Chart
        chart_path = os.path.join(PROJECT_FOLDER, chart_file)
        chart_img = load_image(chart_path)
        if chart_img:
            elements.append(chart_img)
            elements.append(Spacer(1, 20))
        else:
            elements.append(Paragraph("No chart available for this phase.", styles["MissingStyle"]))

        elements.append(PageBreak())

    # Conclusion
    elements.append(Paragraph("Conclusion", styles["HeaderStyle"]))
    elements.append(Paragraph(
        "This case study compiles all completed phases of the FloSports Data Insights project. "
        "Missing sections indicate areas for future work or ongoing analysis. "
        "The combination of exploratory analysis, experimentation, forecasting, and survey insights "
        "demonstrates a broad range of data analysis skills relevant to sports analytics.",
        styles["BodyStyle"]
    ))

    doc.build(elements)
    print(f"✅ Case study PDF created: {OUTPUT_FILE}")

if __name__ == "__main__":
    build_pdf()


✅ Case study PDF created: ./output\FloSports_Case_Study.pdf


## Full Phase 5 Result Visualization

In [2]:
# phase5_survey_analysis.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
import os
import random
from datetime import datetime, timedelta

# -----------------
# 1. Setup Folders
# -----------------
data_folder = "data"
output_folder = "output"
os.makedirs(data_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# -----------------
# 2. Generate Synthetic Survey Data
# -----------------
np.random.seed(42)
customer_ids = [f"CUST_{i:04d}" for i in range(1, 201)]

nps_categories = ["Promoter", "Passive", "Detractor"]
nps_scores = {"Promoter": (9, 10), "Passive": (7, 8), "Detractor": (0, 6)}

feedback_samples = {
    "Promoter": [
        "Amazing service, I watch every week!",
        "Great coverage, worth every cent.",
        "Love the commentary and highlights.",
        "Fantastic app experience."
    ],
    "Passive": [
        "Service is okay, could be improved.",
        "Content is fine but needs more variety.",
        "Neutral experience overall.",
        "Streaming quality is inconsistent."
    ],
    "Detractor": [
        "Too expensive for the value.",
        "Frequent buffering ruins the experience.",
        "Content is repetitive and boring.",
        "App crashes often."
    ]
}

rows = []
start_date = datetime(2024, 1, 1)

for cust in customer_ids:
    category = np.random.choice(nps_categories, p=[0.5, 0.3, 0.2])
    score_range = nps_scores[category]
    nps_score = np.random.randint(score_range[0], score_range[1] + 1)
    csat_score = np.random.randint(1, 6)
    feedback = random.choice(feedback_samples[category])
    survey_date = start_date + timedelta(days=np.random.randint(0, 365))

    rows.append([cust, csat_score, category, nps_score, feedback, survey_date.strftime("%d/%m/%Y")])

df = pd.DataFrame(rows, columns=["CustomerID", "CSAT_Score", "NPS_Category", "NPS_Score_Numeric", "Open_Ended_Feedback", "Survey_Date"])
csv_path = os.path.join(data_folder, "synthetic_sports_survey_responses.csv")
df.to_csv(csv_path, index=False, encoding="utf-8")

# -----------------
# 3. Create Category-Colored Word Clouds
# -----------------
category_colors = {
    "Promoter": "green",
    "Passive": "gold",
    "Detractor": "red"
}

plt.figure(figsize=(12, 8))
for idx, category in enumerate(nps_categories, 1):
    text = " ".join(df[df["NPS_Category"] == category]["Open_Ended_Feedback"])
    wc = WordCloud(width=800, height=400, background_color="white", colormap=None, color_func=lambda *args, **kwargs: category_colors[category]).generate(text)

    plt.subplot(3, 1, idx)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"{category} Feedback Word Cloud", fontsize=14, color=category_colors[category])

chart_path = os.path.join(output_folder, "phase5_wordclouds.png")
plt.tight_layout()
plt.savefig(chart_path, dpi=300)
plt.close()

# -----------------
# 4. Business Insights
# -----------------
promoters_pct = (df["NPS_Category"] == "Promoter").mean() * 100
passives_pct = (df["NPS_Category"] == "Passive").mean() * 100
detractors_pct = (df["NPS_Category"] == "Detractor").mean() * 100
avg_csat = df["CSAT_Score"].mean()

business_summary = f"""
Survey Analysis Summary:
- Promoters: {promoters_pct:.1f}%
- Passives: {passives_pct:.1f}%
- Detractors: {detractors_pct:.1f}%
- Average CSAT Score: {avg_csat:.2f}/5

Business Interpretation:
The high percentage of Promoters suggests strong brand loyalty, but the {detractors_pct:.1f}% detractor rate signals risk of churn.
Passives present a clear upsell opportunity, as they are neutral but not enthusiastic.
Improving streaming quality, adding fresh content, and optimizing pricing can convert Passives and reduce Detractors.
"""

txt_path = os.path.join(output_folder, "phase5_summary.txt")
with open(txt_path, "w", encoding="utf-8") as f:
    f.write(business_summary)

# -----------------
# 5. Generate PDF Report
# -----------------
pdf_path = os.path.join(output_folder, "phase5_survey_report.pdf")
doc = SimpleDocTemplate(pdf_path, pagesize=A4)
styles = getSampleStyleSheet()
story = []

story.append(Paragraph("Phase 5: Survey Analysis Report", styles["Title"]))
story.append(Spacer(1, 12))
story.append(Paragraph(business_summary.replace("\n", "<br/>"), styles["Normal"]))
story.append(Spacer(1, 20))

story.append(Paragraph("Category-Based Feedback Word Clouds", styles["Heading2"]))
story.append(Spacer(1, 12))
story.append(Image(chart_path, width=500, height=600))

doc.build(story)

print(f"✅ Phase 5 completed! Data saved to {csv_path}, visuals to {chart_path}, PDF to {pdf_path}")


✅ Phase 5 completed! Data saved to data\synthetic_sports_survey_responses.csv, visuals to output\phase5_wordclouds.png, PDF to output\phase5_survey_report.pdf


Alright — here’s your **full Phase 5 script** for generating realistic synthetic survey data, category-colored visuals, business-focused insights, and a polished PDF report.

Save this as `phase5_survey_analysis.py` in your project folder and run it once.

```python
# phase5_survey_analysis.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
import os
import random
from datetime import datetime, timedelta

# -----------------
# 1. Setup Folders
# -----------------
data_folder = "data"
output_folder = "output"
os.makedirs(data_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# -----------------
# 2. Generate Synthetic Survey Data
# -----------------
np.random.seed(42)
customer_ids = [f"CUST_{i:04d}" for i in range(1, 201)]

nps_categories = ["Promoter", "Passive", "Detractor"]
nps_scores = {"Promoter": (9, 10), "Passive": (7, 8), "Detractor": (0, 6)}

feedback_samples = {
    "Promoter": [
        "Amazing service, I watch every week!",
        "Great coverage, worth every cent.",
        "Love the commentary and highlights.",
        "Fantastic app experience."
    ],
    "Passive": [
        "Service is okay, could be improved.",
        "Content is fine but needs more variety.",
        "Neutral experience overall.",
        "Streaming quality is inconsistent."
    ],
    "Detractor": [
        "Too expensive for the value.",
        "Frequent buffering ruins the experience.",
        "Content is repetitive and boring.",
        "App crashes often."
    ]
}

rows = []
start_date = datetime(2024, 1, 1)

for cust in customer_ids:
    category = np.random.choice(nps_categories, p=[0.5, 0.3, 0.2])
    score_range = nps_scores[category]
    nps_score = np.random.randint(score_range[0], score_range[1] + 1)
    csat_score = np.random.randint(1, 6)
    feedback = random.choice(feedback_samples[category])
    survey_date = start_date + timedelta(days=np.random.randint(0, 365))

    rows.append([cust, csat_score, category, nps_score, feedback, survey_date.strftime("%d/%m/%Y")])

df = pd.DataFrame(rows, columns=["CustomerID", "CSAT_Score", "NPS_Category", "NPS_Score_Numeric", "Open_Ended_Feedback", "Survey_Date"])
csv_path = os.path.join(data_folder, "synthetic_sports_survey_responses.csv")
df.to_csv(csv_path, index=False, encoding="utf-8")

# -----------------
# 3. Create Category-Colored Word Clouds
# -----------------
category_colors = {
    "Promoter": "green",
    "Passive": "gold",
    "Detractor": "red"
}

plt.figure(figsize=(12, 8))
for idx, category in enumerate(nps_categories, 1):
    text = " ".join(df[df["NPS_Category"] == category]["Open_Ended_Feedback"])
    wc = WordCloud(width=800, height=400, background_color="white", colormap=None, color_func=lambda *args, **kwargs: category_colors[category]).generate(text)

    plt.subplot(3, 1, idx)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"{category} Feedback Word Cloud", fontsize=14, color=category_colors[category])

chart_path = os.path.join(output_folder, "phase5_wordclouds.png")
plt.tight_layout()
plt.savefig(chart_path, dpi=300)
plt.close()

# -----------------
# 4. Business Insights
# -----------------
promoters_pct = (df["NPS_Category"] == "Promoter").mean() * 100
passives_pct = (df["NPS_Category"] == "Passive").mean() * 100
detractors_pct = (df["NPS_Category"] == "Detractor").mean() * 100
avg_csat = df["CSAT_Score"].mean()

business_summary = f"""
Survey Analysis Summary:
- Promoters: {promoters_pct:.1f}%
- Passives: {passives_pct:.1f}%
- Detractors: {detractors_pct:.1f}%
- Average CSAT Score: {avg_csat:.2f}/5

Business Interpretation:
The high percentage of Promoters suggests strong brand loyalty, but the {detractors_pct:.1f}% detractor rate signals risk of churn.
Passives present a clear upsell opportunity, as they are neutral but not enthusiastic.
Improving streaming quality, adding fresh content, and optimizing pricing can convert Passives and reduce Detractors.
"""

txt_path = os.path.join(output_folder, "phase5_summary.txt")
with open(txt_path, "w", encoding="utf-8") as f:
    f.write(business_summary)

# -----------------
# 5. Generate PDF Report
# -----------------
pdf_path = os.path.join(output_folder, "phase5_survey_report.pdf")
doc = SimpleDocTemplate(pdf_path, pagesize=A4)
styles = getSampleStyleSheet()
story = []

story.append(Paragraph("Phase 5: Survey Analysis Report", styles["Title"]))
story.append(Spacer(1, 12))
story.append(Paragraph(business_summary.replace("\n", "<br/>"), styles["Normal"]))
story.append(Spacer(1, 20))

story.append(Paragraph("Category-Based Feedback Word Clouds", styles["Heading2"]))
story.append(Spacer(1, 12))
story.append(Image(chart_path, width=500, height=600))

doc.build(story)

print(f"✅ Phase 5 completed! Data saved to {csv_path}, visuals to {chart_path}, PDF to {pdf_path}")
```

---

### What this script does:

1. **Generates** realistic synthetic survey data (`synthetic_sports_survey_responses.csv`)
2. **Creates** category-based colored word clouds (Promoter → green, Passive → yellow, Detractor → red)
3. **Calculates** business KPIs and writes them in a clear interpretation section
4. **Produces** a clean PDF report with visuals and insights


## Phase 6: Master Portfolio

What this script does (high level)

Builds a polished intro / executive section (cover page, TOC, executive summary, per-phase short blocks with images) using reportlab.

Then merges that intro with any existing full-phase PDFs (Phase 3 A/B PDF, Phase 4 Forecast PDF, Phase 5 Survey PDF, etc.) using PyPDF2, producing one FloSports_Master_Portfolio.pdf.

Is fallback-ready: if summary text, charts, or full PDFs are missing it inserts a clear placeholder and keeps running.

Keeps everything ASCII-safe and professional-looking.

In [3]:
#!/usr/bin/env python3
"""
build_master_portfolio.py

Creates a master portfolio PDF by:
  1) Generating an intro/executive section (cover, TOC, per-phase summaries + visuals) with reportlab
  2) Merging that intro PDF with any existing full-phase PDFs (if present) using PyPDF2

Place phase text files and charts in ./output as:
  - phase1_summary.txt, phase1_chart.png (optional)
  - phase2_summary.txt, phase2_chart.png
  - phase3_summary.txt, phase3_chart.png, (phase3_full.pdf optional)
  - phase4_summary.txt, phase4_chart.png, (phase4_full.pdf optional)
  - phase5_summary.txt, phase5_chart.png, (phase5_full.pdf optional)

Run:
  python build_master_portfolio.py
"""
import os
from datetime import datetime
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.units import mm
from reportlab.platypus import (SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak, Table,
                                TableStyle, Frame)
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from PyPDF2 import PdfMerger
from PIL import Image as PILImage

# ---------- CONFIG ----------
OUTPUT_DIR = "output"
INTRO_PDF = os.path.join(OUTPUT_DIR, "master_intro.pdf")
MASTER_PDF = os.path.join(OUTPUT_DIR, "FloSports_Master_Portfolio.pdf")

PHASES = [
    {"key": "phase1", "title": "Phase 1 — Project Setup", "summary": "phase1_summary.txt", "chart": "phase1_chart.png", "full_pdf": "phase1_full.pdf"},
    {"key": "phase2", "title": "Phase 2 — Exploratory Data Analysis", "summary": "phase2_summary.txt", "chart": "phase2_chart.png", "full_pdf": "phase2_full.pdf"},
    {"key": "phase3", "title": "Phase 3 — A/B Testing", "summary": "phase3_summary.txt", "chart": "phase3_chart.png", "full_pdf": "AB_Test_Report.pdf"},
    {"key": "phase4", "title": "Phase 4 — Demand Forecasting", "summary": "phase4_summary.txt", "chart": "phase4_chart.png", "full_pdf": "Demand_Forecast_Report.pdf"},
    {"key": "phase5", "title": "Phase 5 — Survey Analysis", "summary": "phase5_summary.txt", "chart": "phase5_chart.png", "full_pdf": "phase5_survey_report.pdf"},
]

# Visual sizing
PAGE_WIDTH, PAGE_HEIGHT = A4
IMAGE_MAX_WIDTH = PAGE_WIDTH - 40*mm
IMAGE_MAX_HEIGHT = 100*mm

# Ensure output dir exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------- Helpers ----------
def read_text(path):
    if not path or not os.path.exists(path):
        return None
    with open(path, "r", encoding="utf-8") as f:
        return f.read().strip()

def find_file(fname):
    path = os.path.join(OUTPUT_DIR, fname)
    return path if os.path.exists(path) else None

def safe_image(path, max_w=IMAGE_MAX_WIDTH, max_h=IMAGE_MAX_HEIGHT):
    """Return a reportlab Image scaled to fit within max_w x max_h, or None if not found."""
    if not path or not os.path.exists(path):
        return None
    try:
        pil = PILImage.open(path)
        w, h = pil.size
        # convert px to points assuming 72 dpi (reportlab) but keep ratio: we'll scale by ratio
        # actual DPI not critical; use proportional scaling
        ratio = min(max_w / w, max_h / h, 1.0)
        draw_w = w * ratio
        draw_h = h * ratio
        return Image(path, width=draw_w, height=draw_h)
    except Exception as e:
        print("Warning: could not open image", path, ":", e)
        return None

# ---------- Build intro PDF ----------
def build_intro_pdf():
    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(name="TitleLarge", fontSize=24, leading=28, spaceAfter=14, textColor=colors.HexColor("#0A2342")))
    styles.add(ParagraphStyle(name="SubTitle", fontSize=12, leading=14, textColor=colors.HexColor("#00A7A7")))
    styles.add(ParagraphStyle(name="SectionHeader", fontSize=14, leading=16, textColor=colors.HexColor("#F25C05"), spaceBefore=12, spaceAfter=6))
    styles.add(ParagraphStyle(name="BodySmall", fontSize=10.5, leading=14))
    styles.add(ParagraphStyle(name="Missing", fontSize=10.5, leading=14, textColor=colors.red))

    doc = SimpleDocTemplate(INTRO_PDF, pagesize=A4,
                            rightMargin=30, leftMargin=30, topMargin=30, bottomMargin=30)
    story = []

    # Cover
    story.append(Paragraph("FloSports Data Insights — Case Study", styles["TitleLarge"]))
    story.append(Paragraph("Portfolio Project by [Your Name]", styles["SubTitle"]))
    story.append(Spacer(1, 12))
    story.append(Paragraph(f"Generated: {datetime.now().strftime('%B %d, %Y')}", styles["BodySmall"]))
    story.append(Spacer(1, 18))
    story.append(Paragraph("Executive Summary", styles["SectionHeader"]))
    # Executive summary comes from phase1_summary.txt if present
    exec_text = read_text(find_file("phase1_summary.txt"))
    if exec_text:
        story.append(Paragraph(exec_text.replace("\n", "<br/>"), styles["BodySmall"]))
    else:
        story.append(Paragraph("No executive summary available yet. Run Phase 1 to add an executive summary.", styles["Missing"]))
    story.append(PageBreak())

    # Table of contents
    story.append(Paragraph("Table of Contents", styles["SectionHeader"]))
    toc_lines = []
    for i, p in enumerate(PHASES, start=1):
        toc_lines.append(f"{i}. {p['title']}")
    for line in toc_lines:
        story.append(Paragraph(line, styles["BodySmall"]))
    story.append(PageBreak())

    # Per-phase short sections
    for i, p in enumerate(PHASES, start=1):
        story.append(Paragraph(f"{i}. {p['title']}", styles["SectionHeader"]))

        # Summary text
        summary_path = find_file(p["summary"])
        text = read_text(summary_path)
        if text:
            story.append(Paragraph(text.replace("\n", "<br/>"), styles["BodySmall"]))
        else:
            story.append(Paragraph("Summary not available for this phase yet.", styles["Missing"]))

        story.append(Spacer(1, 8))

        # Chart image
        chart_path = find_file(p["chart"])
        img = safe_image(chart_path)
        if img:
            story.append(img)
            story.append(Spacer(1, 10))
        else:
            story.append(Paragraph("Chart not available for this phase.", styles["Missing"]))

        # If there is a short "recommendation" file (e.g., phaseX_recs.txt), include it
        rec_path = find_file(p["key"] + "_recs.txt") if "key" in p else None
        if rec_path and os.path.exists(rec_path):
            rec_text = read_text(rec_path)
            story.append(Paragraph("<b>Recommendations</b>", styles["BodySmall"]))
            story.append(Paragraph(rec_text.replace("\n", "<br/>"), styles["BodySmall"]))

        story.append(PageBreak())

    # Conclusion
    story.append(Paragraph("Conclusion & Next Steps", styles["SectionHeader"]))
    completed = []
    missing = []
    for p in PHASES:
        full_pdf_path = find_file(p["full_pdf"])
        if full_pdf_path:
            completed.append(p["title"])
        else:
            missing.append(p["title"])
    con_text = "Completed analyses: " + (", ".join(completed) if completed else "None") + ".\n"
    con_text += "Phases still missing full reports: " + (", ".join(missing) if missing else "None") + ".\n"
    con_text += "Recommendation: add missing phase outputs and re-run this builder to include them in the master portfolio."
    story.append(Paragraph(con_text.replace("\n", "<br/>"), styles["BodySmall"]))

    doc.build(story)
    print("Intro PDF built:", INTRO_PDF)

# ---------- Merge into master ----------
def merge_with_phase_pdfs():
    merger = PdfMerger()
    # 1) intro
    if os.path.exists(INTRO_PDF):
        merger.append(INTRO_PDF)
    # 2) append any full-phase PDFs if present (keeps original phase pdf pages)
    for p in PHASES:
        full_pdf_name = p.get("full_pdf")
        if not full_pdf_name:
            continue
        full_pdf_path = find_file(full_pdf_name)
        if full_pdf_path:
            print("Appending full phase PDF:", full_pdf_path)
            merger.append(full_pdf_path)
        else:
            print("No full PDF found for", p["title"], "- skipping.")
    # 3) write out
    with open(MASTER_PDF, "wb") as fout:
        merger.write(fout)
    merger.close()
    print("Master portfolio PDF created:", MASTER_PDF)

# ---------- Run ----------
if __name__ == "__main__":
    build_intro_pdf()
    merge_with_phase_pdfs()
    print("Done. Open", MASTER_PDF)


Intro PDF built: output\master_intro.pdf
No full PDF found for Phase 1 — Project Setup - skipping.
No full PDF found for Phase 2 — Exploratory Data Analysis - skipping.
No full PDF found for Phase 3 — A/B Testing - skipping.
Appending full phase PDF: output\Demand_Forecast_Report.pdf
Appending full phase PDF: output\phase5_survey_report.pdf
Master portfolio PDF created: output\FloSports_Master_Portfolio.pdf
Done. Open output\FloSports_Master_Portfolio.pdf
